# Reinforcement Learning

 Para o aprendizado por reforço foi escolhido o método Q-Learning para ser utilizado. Um método model-free e value-based, onde inicialmente iremos definir uma Q-Table ($Q[S, A]$) com conjunto de estados e ações e atualizando o valor de qualidade para cada $Q(s,a)$ ao decorrer do treinamento através de recompensa e a Q-function. A seguir será explicado a teoria utilizada e o modelo a ser implementado, após isso será apresentado o código e explicações das funções mais relevantes e por fim os gráficos gerados e conclusões do processo. Este segmento, assim como o modelo genético, não poderá ser executado, apesar de pré-gerados os códigos e resultados serão todos apresentados em markdown.

# Passos do Q-Learning

## Passo 1: Q-Table  

A tabela utilizada no Q-Learning é uma combinação de estados $s$ do conjunto $S$ e ações $a$ do conjunto $A$, sendo assim, cada $(s,a)$ possui um valor de qualidade dado por $Q(s,a)$.   
  
O ideal seria utilizar o estado completo do jogo, porém para poupar poder computacional, construímos os estados baseado em quais ações são permitidas na atual posição do pacman (Norte, Sul, Leste e Oeste), se possui algum fantasma perto (menor que 3.5 unidades de distância), posições relativas do fantasma e da comida mais próxima (Norte, Sul, Leste, Oeste, Nordeste, Sudeste, Noroeste e Sudoeste). O conjunto de ações consiste em Norte, Sul, Leste e Oeste.  
  
Sendo assim, obteremos uma tabela com a coluna sendo os estados e cada linha sendo as ações. O modelo irá analisar a cada nova posição do pacman qual estado ele está, se ainda não estiver na tabela ele é incluído e todos os elementos terão valor inicialmente 0.
  


## Passo 2: Escolhendo e perfomando a ação

A cada estado deve-se escolher uma ação para depois ser avaliada e o intuito é escolher a ação que maximizará o valor de Q para o estado atual, porém deve-se balancear o exploration e exploitation do agente, sendo assim o modelo possui um $\epsilon = 0.1$ que o fornece a probabilidade de 10% de escolher uma ação aleatória permitida naquele estado, evitando assim que sempre repita uma ação, mesmo que boa, pois há a possibilidade de uma melhor, ainda não explorada, existir.


## Passo 3: Medindo a recompensa

Para avaliar a ação e atualizar o valor em $Q(s,a)$ precisa-se definir uma recompensa para ação. A função de recompensa $R(s,a)$ para esse modelo se baseia no seguinte algoritmo:
* Se possuía um fantasma próximo:
    * Se a distância aumentou +30 pontos
    * Senão -30 pontos
* Senão:
    * Se comeu comida ou cápsula:
        * Se comeu comida +10 pontos
        * Se comeu cápsula +20 pontos
    * Senão:
        * Se aproximou da comida +15 pontos
        * Senão -15 pontos



## Passo 4: Atualizando a Q-Table

Cada elemento da Q-Table será atualizada através da equação de Bellman:  
  
$New Q(s,a) = Q(s,a) + \alpha(R(s,a) + \gamma maxQ(s,a) - Q(s,a))$  
  
Onde:  
$New Q(s,a)$ é o novo elemento em $Q(s,a)$.  
$Q(s,a)$ é o valor atual em Q-Table.  
$\alpha$ é o learning rate, nosso modelo será 0.2.  
$R(s,a)$ é a função de recompensa.  
$\gamma$ é o discount rate, nosso modelo será 0.8.  
$maxQ(s,a)$ é o maior Q-value associado a este estado.  

## Passo 5: Repetir os passos 2,3 e 4

O modelo deverá executar diversos episódios até aprender, idealmente poderíamos definir um critério de parada para o algoritmo, porém como iremos passar o número de treinos a ser realizado por linha de comando tivemos dificuldade em definir um critério de parada, sendo assim, para o layout de smallClassic será executado 10 mil episódios, para o mediumClassic 5 mil e o originalClassic 2mil. O número de treinos foi se reduzindo por limitações computacionais.

# Implementação do Q-learning

A implementação segue o seguinte algoritmo:

* No construtor:
    * Inicializa um dicionário para armazenar $Q[S,A]$
    * Inicializa os parâmetros utilizados na Q-Function
    * Inicializa atributos utilizados para avaliar a recompensa  
  
  
* Função getAction():
    * Analisa qual o estado atual
    * Avalia a recompensa
    * Atualiza Q-Table
    * Atualiza os atributos da classe
    * Escolhe uma nova ação
  

* Função final():
    * Analisa qual o estado atual
    * Avalia a recompensa
    * Atualiza Q-Table 
    * Reseta os atributos da classe
    * Finaliza o episódio


No arquivo Agents.py possui a implementação da classe QLearnAgent que será apresentada e explicada aqui. Segue abaixo o código completo e a explicação para as principais funções

Inicialmente temos o construtor da classe incializando a Q-Table com um dicionário e definindo outros atributos que serão utilizados.

```python
class QLearnAgent(Agent):

    # Inicializa o Agent com os atributos necessários
    def __init__(self, numTraining = 10):

        # Parâmetros do Q-Learning 
        self.alpha = 0.2
        self.epsilon = 0.1
        self.gamma = 0.8
        self.qValues = dict()

        # Acompanha resultados durante o episódio
        self.numTraining = int(numTraining)
        self.episodesSoFar = 0
        self.actionsSoFar = 0
        self.totalReward = 0
        
        # Usado para criar estado e recompensas
        self.lastState = None
        self.lastAction = None
        self.lastScore = None
        self.lastNumFood = None
        self.lastCaps = None
        self.lastDistGhost = None
        self.lastDistFood = None
        self.doNotEat = None
        self.ghostWasNear = False
        self.lastFoodPosition = None

        # Escreve em arquivos diferente dependendo do número de treino
        if(self.getNumTraining() < 2500):
            with open('episodesResults3.txt','w') as f:
                f.write("")
        elif(self.getNumTraining() < 5500):
            with open('episodesResults2.txt','w') as f:
                f.write("")
        else:
            with open('episodesResults1.txt','w') as f:
                f.write("")
```
A seguir são algumas funções para acessar, setar e incrementar os atributos.

```python

    # Funções para acessar os atributos

    def incrementEpisodesSoFar(self):
        self.episodesSoFar += 1

    def getEpisodesSoFar(self):
        return self.episodesSoFar
    
    def incrementAcionsSoFar(self):
        self.actionsSoFar += 1
    
    def getAcionsSoFar(self):
        return self.actionsSoFar

    def getNumTraining(self):
        return self.numTraining

    def setEpsilon(self, value):
        self.epsilon = value

    def getAlpha(self):
        return self.alpha

    def setAlpha(self, value):
        self.alpha = value

    def getGamma(self):
        return self.gamma
```

A função seguinte recebe o estado do jogo e os movimentos legais do pacman e retorna uma string descrevendo o estado do conjunto S

```python
    # Cria o estado baseado em direções legais, distância do fantasma mais próximo e sua posição relativa e da comida mais próxima
    def createState(self,state,legal):
        posG = np.array(state.getGhostPositions())
        posP = np.array(state.getPacmanPosition())
        posFood = state.getFood()
        numFood = state.getNumFood()

        minDistGhost = 99999
        indG = None
        for i in range(len(posG)):
            if(np.linalg.norm(posG[i]-posP) < minDistGhost):
                minDistGhost = np.linalg.norm(posG[i]-posP)
                indG =  i

        if(posG[indG][0] > posP[0]):
            if(posG[indG][1] > posP[1]):
                ghostRelativePosition = "upright"
            elif(posG[indG][1] == posP[1]):
                ghostRelativePosition = "right"
            else:
                ghostRelativePosition = "downright"

        elif(posG[indG][0] == posP[0]):
            if(posG[indG][1] > posP[1]):
                ghostRelativePosition = "up"
            else:
                ghostRelativePosition = "down"
        else:
            if(posG[indG][1] > posP[1]):
                ghostRelativePosition = "upleft"
            elif(posG[indG][1] == posP[1]):
                ghostRelativePosition = "left"
            else:
                ghostRelativePosition = "downleft"

        if(numFood > 0):
            minDistFood = 99999
            minFood = None
            for x in range(posFood.width):
                for y in range(posFood.height):
                    if(posFood[x][y]):
                        food = np.array([x,y])
                        distFood = np.linalg.norm(food-posP)
                        if(distFood<minDistFood):
                            minDistFood = distFood
                            minFood = food
            
            if(minFood[0] > posP[0]):
                if(minFood[1] > posP[1]):
                    foodRelativePosition = "upright"
                elif(minFood[1] == posP[1]):
                    foodRelativePosition = "right"
                else:
                    foodRelativePosition = "downright"

            elif(minFood[0] == posP[0]):
                if(minFood[1] > posP[1]):
                    foodRelativePosition = "up"
                else:
                    foodRelativePosition = "down"
            else:
                if(minFood[1] > posP[1]):
                    foodRelativePosition = "upleft"
                elif(minFood[1] == posP[1]):
                    foodRelativePosition = "left"
                else:
                    foodRelativePosition = "downleft"
            self.lastFoodPosition  = foodRelativePosition
        else:
            foodRelativePosition = self.lastFoodPosition

        if(minDistGhost < 3):
            ghostNear = True
        else:
            ghostNear = False
        
        # Retorna uma string descrevendo o estado
        return str(legal)+str(ghostNear)+str(ghostRelativePosition)+str(foodRelativePosition)
```
A função seguinte inicializa um novo Q(s,a)

```python

    # Caso o estado ainda não exista é gerado um para cada ação legal com valor Q = 0
    def initializeQValues(self, pacmanState, legal):
        self.qValues[pacmanState] = dict()
        for action in legal:
            if action not in self.qValues[pacmanState]:
                self.qValues[pacmanState][action] = 0.0
```
A seguinte função é utilizada para avaliar a recompensa da última ação e atualizar o valor na Q-Table
```python

    # Calcula a recompensa da última ação e atualiza  valor Q do estado e sua ação através da função do Q-learning
    def updateQValue(self, state,pacmanState, final_step=False):
        
        posG = np.array(state.getGhostPositions())
        posP = np.array(state.getPacmanPosition())
        posFood = state.getFood()

        numFoodEat = self.lastNumFood - state.getNumFood()
        numCapsEat = self.lastCaps - len(state.getCapsules())

        minDistGhost = 99999
        for i in range(len(posG)):
            if(np.linalg.norm(posG[i]-posP) < minDistGhost):
                minDistGhost = np.linalg.norm(posG[i]-posP)
        
        minDistFood = 99999
        for x in range(posFood.width):
            for y in range(posFood.height):
                if(posFood[x][y]):
                    food = np.array([x,y])
                    distFood = np.linalg.norm(food-posP)
                    if(distFood<minDistFood):
                        minDistFood = distFood

        if(self.ghostWasNear):
            if(minDistGhost >= self.lastDistGhost):
                didNotEat = 30
            else:
                didNotEat = -30
        else:
            if(numFoodEat == 0 or numCapsEat == 0):
                if(minDistFood > self.lastDistFood):
                    didNotEat = -15
                else:
                    didNotEat = 15
            else:
                didNotEat = 0

        reward = numFoodEat * 10 + numCapsEat * 20 + didNotEat
        self.totalReward += reward

        max_Q_value = 0.0
        if not final_step:
            max_Q_value = max(list(self.qValues[pacmanState].values()))
        self.qValues[self.lastState][self.lastAction] += (self.alpha * (reward + self.gamma * max_Q_value - self.qValues[self.lastState][self.lastAction]))
```
A função seguinte escolhe uma nova ação, com possibilidade de ser uma ação aleatória.

```python

    # Escolha a nova ação balanceando exploration e exploitation
    def epsilonGreedy(self, state,pacmanState, legal):
        if Directions.STOP in legal:
            legal.remove(Directions.STOP)

        if not self.ghostWasNear:
            if(self.lastAction!= None):
                if (Directions.REVERSE[self.lastAction] in legal) and len(legal)>1:
                    legal.remove(Directions.REVERSE[self.lastAction])

        probability = random.random()

        if probability < self.epsilon: # Escolhe ação aleatória
            random_action = random.choice(legal)
            return random_action

        max_Q_action = None # Escolhe a que já sabe ser melhor
        for action in legal:
            if max_Q_action == None:
                max_Q_action = action
            if self.qValues[pacmanState][action] > self.qValues[pacmanState][max_Q_action]:
                max_Q_action = action
        return max_Q_action
```
As duas funções a seguir são somente utilizadas para atualizar ou resetar os atributos da classe

```python

    # Atualiza atributos para um novo estado
    def updateAttributes(self, state,pacmanState, legal):

        posG = np.array(state.getGhostPositions())
        posP = np.array(state.getPacmanPosition())
        posFood = state.getFood()

        self.lastNumFood = state.getNumFood()
        self.lastCaps = len(state.getCapsules())

        minDistGhost = 99999
        for i in range(len(posG)):
            if(np.linalg.norm(posG[i]-posP) < minDistGhost):
                minDistGhost = np.linalg.norm(posG[i]-posP)
        
        minDistFood = 99999
        for x in range(posFood.width):
            for y in range(posFood.height):
                if(posFood[x][y]):
                    food = np.array([x,y])
                    distFood = np.linalg.norm(food-posP)
                    if(distFood<minDistFood):
                        minDistFood = distFood
        
        if(minDistGhost <= 3.5 ):
            self.ghostWasNear = True
        else:
            self.ghostWasNear = False
        self.lastDistGhost = minDistGhost
        self.lastDistFood = minDistFood
        self.lastState = self.createState(state,legal)
        self.lastAction = self.epsilonGreedy(state,pacmanState, legal)
        self.lastScore = state.getScore()

    # Reseta todos atributos (no fim do episódio)
    def resetAttributes(self):
        self.totalReward = 0
        self.actionsSoFar = 0
        self.lastState = None
        self.lastAction = None
        self.lastScore = None
```

A função getAction é chamada a cada passo do pacman e realiza os passos do Q-Learning utilizando as funções já implementadas

```python

    # Escolhe uma ação, inicializa o estado que o pacman está caso não exista ainda e move o pacman
    def getAction(self, state):

        legal = state.getLegalPacmanActions()

        if Directions.STOP in legal: # Remove a ação de parar
            legal.remove(Directions.STOP)
        
        if not self.ghostWasNear: # Se não possuir fantasmas próximos impede o pacman reverter
            if(self.lastAction!= None):
                if (Directions.REVERSE[self.lastAction] in legal) and len(legal)>1:
                    legal.remove(Directions.REVERSE[self.lastAction])
        
        pacmanState = self.createState(state,legal) # Analisa o estado atual

        if pacmanState not in self.qValues: # Insere o estado na Q-Table
            self.initializeQValues(pacmanState, legal)

        if self.lastState != None: # Avalia a recompensa e atualiza Q-Table
            self.updateQValue(state,pacmanState)

        self.updateAttributes(state,pacmanState, legal) # Atualiza os atributos da classe

        self.incrementAcionsSoFar()

        return self.lastAction
```
A função final é chamada após uma derrota ou vitória encerrando o episódio

```python

    # Chamada no fim do episódio e atualiza o último estágio
    def final(self, state):
        legal = state.getLegalPacmanActions()

        if Directions.STOP in legal:
            legal.remove(Directions.STOP)
        
        if not self.ghostWasNear:
            if(self.lastAction!= None):
                if (Directions.REVERSE[self.lastAction] in legal) and len(legal)>1:
                    legal.remove(Directions.REVERSE[self.lastAction])

        if self.lastState != None:
            self.updateQValue(state,self.createState(state,legal), final_step=True)


        if(self.getNumTraining() < 2500):
            with open('episodesResults3.txt','a') as f:
                f.write(str(self.totalReward)+" "+str(self.getAcionsSoFar())+" "+str(state.getScore())+"\n")
        elif(self.getNumTraining() < 5500):
            with open('episodesResults2.txt','a') as f:
                f.write(str(self.totalReward)+" "+str(self.getAcionsSoFar())+" "+str(state.getScore())+"\n")
        else:
            with open('episodesResults1.txt','a') as f:
                f.write(str(self.totalReward)+" "+str(self.getAcionsSoFar())+" "+str(state.getScore())+"\n")

        self.resetAttributes()

        self.incrementEpisodesSoFar()

        if self.getEpisodesSoFar() == self.getNumTraining():
            self.setAlpha(0)
            self.setEpsilon(0)


```

# Execução do Q-Learning

Para executar os treinos foi utilizado o seguinte código num notebook, o tempo de execução foi 1.84 horas.  
Assim foi escrito em arquivos separados os resultados de cada episódio e também os resultados das 10 execuções dos melhores agentes.


``` python
saida1 = !python pacman.py -p QLearnAgent -x 10000 -n 10010 -l smallClassic 
with open('results1.txt','w') as f:
    for i in saida1:
        f.write(str(i)+'\n')

saida2 = !python pacman.py -p QLearnAgent -x 5000 -n 5010 -l mediumClassic 
with open('results2.txt','w') as f:
    for i in saida2:
        f.write(str(i)+'\n')

saida3 = !python pacman.py -p QLearnAgent -x 2000 -n 2010 -l originalClassic 
with open('results3.txt','w') as f:
    for i in saida3:
        f.write(str(i)+'\n')
```

A seguir segue o código excutado para gerar os gráficos

``` python
import matplotlib.pyplot as plt
from matplotlib import ticker

def createGraphs(file,title,YLabel,EpNum,ind,fileGenerated):
    with open(file) as f:
        lines = f.readlines()
        xy = [[int(line.split()[0]),int(line.split()[1]),float(line.split()[2])] for line in lines]

    epResults = []
    for i in range(len(xy[0])):  
        aux1 = []
        eps = []
        for j in range(len(xy)):
            eps.append(j+1)
            aux1.append(xy[j][i])
        epResults.append(aux1)

    fig = plt.figure(figsize=(60, 6))
    ax = fig.add_subplot()
    ax = plt.axes()
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1000))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(250))
    plt.xlim(0, EpNum)
    plt.scatter(eps,epResults[ind])
    plt.tick_params(axis='x', which='major', labelsize=30)
    plt.tick_params(axis='y', which='major', labelsize=30)
    plt.title(title,fontsize=30)
    plt.xlabel('Episodes',fontsize=30)
    plt.ylabel(YLabel,fontsize=30)
    plt.savefig(fileGenerated)

createGraphs("episodesResults1.txt","Reward x Episodes smallClassic","Total Reward",10000,0,"TRsmall.png")
createGraphs("episodesResults1.txt","Actions x Episodes smallClassic","Total Actions",10000,1,"TAsmall.png")
createGraphs("episodesResults1.txt","Score x Episodes smallClassic","Total Score",10000,2,"TSsmall.png")

createGraphs("episodesResults2.txt","Reward x Episodes mediumClassic","Total Reward",5000,0,"TRmedium.png")
createGraphs("episodesResults2.txt","Actions x Episodes mediumClassic","Total Actions",5000,1,"TAmedium.png")
createGraphs("episodesResults2.txt","Score x Episodes mediumClassic","Total Score",5000,2,"TSmedium.png")

createGraphs("episodesResults3.txt","Reward x Episodes originalClassic","Total Reward",2000,0,"TRoriginal.png")
createGraphs("episodesResults3.txt","Actions x Episodes originalClassic","Total Actions",2000,1,"TAoriginal.png")
createGraphs("episodesResults3.txt","Score x Episodes originalClassic","Total Score",2000,2,"TSoriginal.png")
```

# Resultados do Q-Learning

## Layout smallClassic

![image](TRsmall.png)
![image](TAsmall.png)
![image](TSsmall.png)

Pacman emerges victorious! Score: 921  
Pacman died! Score: -347  
Pacman emerges victorious! Score: 1137  
Pacman emerges victorious! Score: 893  
Pacman died! Score: -214  
Pacman emerges victorious! Score: 967  
Pacman died! Score: -109  
Pacman emerges victorious! Score: 938  
Pacman emerges victorious! Score: 931  
Pacman emerges victorious! Score: 884  
Average Score: 600.1  
Scores: 921.0, -347.0, 1137.0, 893.0, -214.0, 967.0, -109.0, 938.0, 931.0, 884.0  
Win Rate: 7/10 (0.70)  
Record: Win, Loss, Win, Win, Loss, Win, Loss, Win, Win, Win  




## Layout mediumClassic

![image](TRmedium.png)
![image](TAmedium.png)
![image](TSmedium.png)

Pacman emerges victorious! Score: 1257  
Pacman emerges victorious! Score: 1320  
Pacman emerges victorious! Score: 1272  
Pacman emerges victorious! Score: 1462  
Pacman died! Score: 171  
Pacman emerges victorious! Score: 1247  
Pacman emerges victorious! Score: 1270  
Pacman emerges victorious! Score: 1317  
Pacman died! Score: -126  
Pacman died! Score: 72  
Average Score: 926.2  
Scores: 1257.0, 1320.0, 1272.0, 1462.0, 171.0, 1247.0, 1270.0, 1317.0, -126.0, 72.0  
Win Rate: 7/10 (0.70)  
Record:  Win, Win, Win, Win, Loss, Win, Win, Win, Loss, Loss  

## Layout originalClassic

![image](TRoriginal.png)
![image](TAoriginal.png)
![image](TSoriginal.png)

Pacman died! Score: 392  
Pacman emerges victorious! Score: 2297  
Pacman died! Score: 615  
Pacman died! Score: 831  
Pacman died! Score: 976  
Pacman died! Score: 293  
Pacman died! Score: 762  
Pacman emerges victorious! Score: 2302  
Pacman died! Score: 885  
Pacman died! Score: 402  
Average Score: 975.5  
Scores: 392.0, 2297.0, 615.0, 831.0, 976.0, 293.0, 762.0, 2302.0, 885.0, 402.0  
Win Rate: 2/10 (0.20)  
Record: Loss, Win, Loss, Loss, Loss, Loss, Loss, Win, Loss, Loss  
